In [3]:
from utils import Ising, conditional, log_unnormalized_p
from plot_utils import *
from tqdm.auto import tqdm

In [4]:
def gibbs_sampling(model, state, n_samples):
    n = model.dim
    model.state = state
    
    samples = []
    states = np.array([[i, j] for i in range(n) for j in range(n)])
    for t in range(n_samples):
        np.random.shuffle(states)
        for i, j in states:
            p_Xij = conditional(model.state, i, j, model.Js, model.Jst)
            model.state[i][j] = np.random.binomial(1, p_Xij) * 2 - 1  # 0 -> -1, 1 -> 1
        samples.append(model.state.copy())

    return samples


In [36]:
def annealed_importance_sampling(p_0, p_G, betas, n_samples, n_steps, return_all_temps=False):
    """
    Annealed Importance Sampling
    
    Args:
        return_all_temps: If True, return estimates for all temperatures
    """
    assert (len(betas) == n_steps + 1)
    
    log_Z_estimates = []
    log_Z_all_temps = [] if return_all_temps else None
    
    target_Js = p_G.Js
    target_Jst = p_G.Jst
    dim = p_G.dim
    
    # for all x, p_0(x) = exp(0) = 1, log(Z_0) = log(2^(dim^2)) = dim^2 * log(2)
    log_Z_0 = dim ** 2 * np.log(2)
    
    for i in tqdm(range(n_samples)):
        sum_log_ratio = log_Z_0
        temp_estimates = [log_Z_0] if return_all_temps else None
        
        for j in range(n_steps):
            if j == 0:
                # Sample x_1 from p_0
                p_0.init_state()
                x_j = p_0.state.copy()
            else:
                # Sample x_{j+1} from p_beta_j given x_j using Gibbs sampling
                p_beta_j = Ising(dim, betas[j] * target_Js, betas[j] * target_Jst)
                x_j = gibbs_sampling(p_beta_j, x_j, 1)[0]
                
            # Compute log ratio: log(p̂_{j+1}(x_{j+1}) / p̂_j(x_{j+1}))
            log_p_j = log_unnormalized_p(x_j, betas[j] * target_Js, betas[j] * target_Jst)
            log_p_j_plus_1 = log_unnormalized_p(x_j, betas[j+1] * target_Js, betas[j+1] * target_Jst)
            
            sum_log_ratio += (log_p_j_plus_1 - log_p_j)
            
            if return_all_temps:
                temp_estimates.append(sum_log_ratio)
            
        log_Z_estimates.append(sum_log_ratio)
        if return_all_temps:
            log_Z_all_temps.append(temp_estimates)
    
    if return_all_temps:
        # Average across samples for each temperature
        log_Z_all_temps = np.array(log_Z_all_temps)  # Shape: (n_samples, n_steps+1)
        log_Z_means = np.mean(log_Z_all_temps, axis=0)
        log_Z_stds = np.std(log_Z_all_temps, axis=0, ddof=1)
        return log_Z_means, log_Z_stds
    else:
        return np.mean(log_Z_estimates), np.std(log_Z_estimates, ddof=1)

In [30]:
def importance_sampling(p_0, p_G, n_samples):
    """
    Standard Importance Sampling to estimate log(Z_G)
    """
    dim = p_G.dim
    log_Z_0 = dim ** 2 * np.log(2)  # Z_0 = 2^(dim^2)
    
    log_weights = []
    for i in tqdm(range(n_samples)):
        p_0.init_state()
        x = p_0.state.copy()
        log_weights.append(log_unnormalized_p(x, p_G.Js, p_G.Jst))
    
    # log(Z_G) = log(Z_0 * mean(exp(log_weights)))
    log_weights = np.array(log_weights)
    max_lw = np.max(log_weights)
    weights = np.exp(log_weights - max_lw)
    
    mean_weight = np.mean(weights)
    log_Z_G = log_Z_0 + max_lw + np.log(mean_weight)
    
    # Standard error of the estimate
    std_weight = np.std(weights, ddof=1) / np.sqrt(n_samples)
    log_Z_G_std = std_weight / mean_weight  # Relative error in log space
    
    return log_Z_G, log_Z_G_std

In [ ]:
# sanity check
dim = 3

target_Js = 0
target_Jst = 1

K = 1000
betas = np.linspace(0, 1, K+1)
M_ais = 1000
M_is = 500000

p_0 = Ising(dim, 0, 0)
p_G = Ising(dim, target_Js, target_Jst)

print(f"Running AIS with M={M_ais} samples, K={K} temperatures...")
ais_mean, ais_std = annealed_importance_sampling(p_0, p_G, betas, M_ais, K)
print(f"AIS - Mean: {ais_mean:.6f}, Std: {ais_std:.6f}")

  0%|          | 0/1000 [00:00<?, ?it/s]

(12.801476625039486, 0.14735476348704288)

In [33]:
print(f"\nRunning IS with {M_is} samples...")
is_mean, is_std = importance_sampling(p_0, p_G, M_is)
print(f"IS  - Mean: {is_mean:.6f}, Std: {is_std:.6f}")


Running IS with 500000 samples...


  0%|          | 0/500000 [00:00<?, ?it/s]

IS  - Mean: 118.390288, Std: 0.828338


In [ ]:
dim = 10

target_Js = 0
target_Jst = 1

K = 1000
betas = np.linspace(0, 1, K+1)
M_ais = 1000
M_is = 500000

p_0 = Ising(dim, 0, 0)
p_G = Ising(dim, target_Js, target_Jst)

print(f"Running AIS with M={M_ais} samples, K={K} temperatures...")
ais_mean, ais_std = annealed_importance_sampling(p_0, p_G, betas, M_ais, K)
print(f"AIS - Mean: {ais_mean:.6f}, Std: {ais_std:.6f}")

  0%|          | 0/1000 [00:00<?, ?it/s]

(180.17941605599447, 1.379343124915689)

In [37]:
print(f"\nRunning IS with {M_is} samples...")
is_mean, is_std = importance_sampling(p_0, p_G, M_is)
print(f"IS  - Mean: {is_mean:.6f}, Std: {is_std:.6f}")


Running IS with 500000 samples...


  0%|          | 0/500000 [00:00<?, ?it/s]

IS  - Mean: 130.192741, Std: 0.999613


In [ ]:
print(f"Difference in means: {abs(ais_mean - is_mean):.6f}")
print(f"AIS std / IS std:    {ais_std / is_std:.6f}")



3. Comparison:


NameError: name 'ais_mean' is not defined